In [1]:
# Copyright 2023 Orange
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Software Name : knowledge-graph-dst
# Version: 1.0.0
# SPDX-FileCopyrightText: Copyright (c) 2023 Orange
# SPDX-License-Identifier: MIT
# This software is distributed under the MIT,
# the text of which is available at https://opensource.org/licenses/MIT
# or see the "LICENSE" file for more details.
# Author: H. Andres Gonzalez
#!pip install bertviz torch transformers ecco
# ecco is broken as hell tho
from transformers import T5ForConditionalGeneration, AutoTokenizer

# BertViz

In [1]:

from bertviz import model_view

In [3]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [4]:




inputs = tokenizer(
    "Translate from English to French: Hello, how are you?",
    return_tensors="pt",
)

output = model.generate(**inputs, return_dict_in_generate=True, output_attentions=True, max_new_tokens=50)

sequences = output.sequences

encoder_attentions = output.encoder_attentions
cross_attentions = output.cross_attentions
decoder_attentions = output.decoder_attentions

print(f"{sequences.shape = }")
print(f"{len(cross_attentions) = }")
print(f"{len(decoder_attentions) = }")

sequences.shape = torch.Size([1, 10])
len(cross_attentions) = 9
len(decoder_attentions) = 9


only some heads and layers
https://github.com/jessevig/bertviz/tree/master#filtering-layers

In [5]:
# Generate
output = model.generate(**inputs, return_dict_in_generate=True, max_new_tokens=50)

# Pass the outputs through the model again to get the attentions
out = model(**inputs, decoder_input_ids=output.sequences, output_attentions=True, return_dict=True)

encoder_attentions = out.encoder_attentions
cross_attentions = out.cross_attentions
decoder_attentions = out.decoder_attentions

print(f"{len(encoder_attentions) = }")
print(f"{len(cross_attentions) = }")
print(f"{len(decoder_attentions) = }")

len(encoder_attentions) = 6
len(cross_attentions) = 6
len(decoder_attentions) = 6


In [6]:
#encoder_text = inputs
encoder_text = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
#inputs['input_ids'][0]
decoder_text = tokenizer.convert_ids_to_tokens(output['sequences'][0])

In [7]:
model_view(
    cross_attention = cross_attentions,
    encoder_attention = encoder_attentions, 
    decoder_attention = decoder_attentions,
    encoder_tokens = encoder_text,
    decoder_tokens = decoder_text)

<IPython.core.display.Javascript object>

In [8]:
from bertviz.neuron_view import show

In [9]:
' '.join(decoder_text)

'<pad> ▁Bonjour , ▁comment ▁ êtes - vous ? </s>'

In [10]:
from bertviz import head_view
head_view(encoder_attentions, encoder_text)

<IPython.core.display.Javascript object>

In [15]:
head_view(decoder_attentions, decoder_text)

<IPython.core.display.Javascript object>

# Ecco viz

https://github.com/jalammar/ecco

https://jalammar.github.io/explaining-transformers/

https://colab.research.google.com/github/jalammar/ecco/blob/main/notebooks/readme.md%20examples.ipynb


In [3]:
!ls ../results/models/Final/t5_base_experiment_1/version_0/checkpoints/best_dst_ckpt

added_tokens.json	pytorch_model.bin	 tokenizer_config.json
config.json		special_tokens_map.json  tokenizer.json
generation_config.json	spiece.model


In [2]:
import ecco

In [3]:
model_path = '../results/models/Final/t5_base_experiment_1/version_0/checkpoints/best_dst_ckpt'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

In [3]:
model.shared  # embedding
model.encoder.embed_tokens
model.decoder.embed_tokens
model.shared.weight

Parameter containing:
tensor([[ -2.0156,   0.2236,  -7.0938,  ...,  -0.3535,   2.6406,  -2.8906],
        [ 12.6250,   8.1875, -11.6250,  ...,   7.9375,  -7.3125,   0.9453],
        [ -8.7500,   7.1875,  27.8750,  ..., -26.7500,   0.8555,  -1.5156],
        ...,
        [-25.2500, -28.5000, -17.2500,  ..., -17.7500,  -5.2500,  27.3750],
        [-25.5000, -29.3750, -18.2500,  ..., -17.7500,  -4.8125,  27.7500],
        [-26.7500, -28.3750, -17.8750,  ..., -18.5000,  -7.0000,  27.6250]],
       requires_grad=True)

In [8]:
# FFN
model.encoder.block[0].layer[1].DenseReluDense.wo

Linear(in_features=2048, out_features=768, bias=False)

In [9]:
# find tokenizer prefix
ids = tokenizer.tokenize("tokenization")
print(ids)

['▁token', 'ization']


In [4]:
# https://github.com/jalammar/ecco/blob/main/src/ecco/model-config.yaml SEE ACTUAL CONFIG
model_config = {
    'embedding': "shared.weight",
    'type': 'enc-dec',
    #'activations': ['intermediate\.dense'], #This is a regex model.encoder.layer[0].intermediate.dense
    #'activations': ['DenseReluDense\.wo'],  # model.encoder.block[0].layer[1].DenseReluDense.wo
    'activations': 'wo', 
    'token_prefix': '▁',
    'partial_token_prefix': ''  # it seems that it is ''? https://github.com/jalammar/ecco/blob/main/notebooks/Identifying%20model%20configuration.ipynb
}

In [5]:
ecco_model = ecco.from_pretrained('t5-small', 
                                  activations=True,
                                  model_config=model_config,
                                  verbose=False)


In [6]:
text = """Please book a taxi for me Sure! when would you like to arrive? I want to arrive by 19:00 I can definitely help! Where are you departing from? 
I am leaving from the Alpha - Milton Guest House. Where will the taxi be taking you tonight? I am eager to visit cambridge university botanic gardens. 
Ok, I've booked your taxi. It will be a grey lexus, contact number 07229148300 What time will the taxi be arriving at the Alpha - Milton? 
You should contact the taxi for an approximate time. search arrive by 19:00 search departure alpha - milton guest house result car grey lexus result phone 07229148300
result type taxi search destination cambridge university botanic gardens search type taxi", "Please book a taxi for me Sure! when would you like to arrive? 
I want to arrive by 19:00 I can definitely help! Where are you departing from? I am leaving from the Alpha - Milton Guest House. Where will the taxi be taking you tonight? ,
I am eager to visit cambridge university botanic gardens. Ok, I've booked your taxi. It will be a grey lexus, contact number 07229148300 
What time will the taxi be arriving at the Alpha - Milton? You should contact the taxi for an approximate time. That is all I need. 
Goodbye. Anytime! Enjoy your stay. search arrive by 19:00 search departure alpha - milton guest house result car grey lexus result phone 07229148300 
search destination cambridge university botanic gardens result type taxi search type taxi
"""

text = "Please book a taxi for me Sure! when would you like to arrive? I want to arrive by 19:00"

In [7]:
help(ecco_model.generate)

Help on method generate in module ecco.lm:

generate(input_str: str, max_new_tokens: Optional[int] = None, min_new_tokens: Optional[int] = 0, max_length: Optional[int] = 20, min_length: Optional[int] = 0, early_stopping: Optional[bool] = False, temperature: Optional[float] = 1.0, top_k: Optional[int] = 50, top_p: Optional[float] = 1.0, do_sample: Optional[bool] = False, attribution: Optional[List[str]] = [], generate: Optional[int] = None, num_beams: int = 1, **other_generate_kwargs: Any) method of ecco.lm.LM instance
    Generate tokens in response to an input prompt.
    Works with Language models like GPT2, not masked language models like BERT.
    Args:
        input_str: Input prompt. # TODO: accept batch of input strings
        generate: Number of tokens to generate.
        max_length: max length of sequence (input + output tokens)
        temperature: Adjust the probability distibution of output candidate tokens.
        top_k: Specify top-k tokens to consider in decoding. Onl

In [69]:
ecco_model

In [5]:
tokenized = tokenizer(text, max_length=512, truncation=True, return_tensors='pt')
args_for_generate = {'max_length': 512, 'min_length': 256, 'early_stopping': True}
args_for_generate.update(tokenized)

In [7]:
args_for_generate.keys()

NameError: name 'args_for_generate' is not defined

In [6]:
example = model._prepare_decoder_input_ids_for_generation(8, model_input_name='input_ids',
                                                          #model_kwargs=args_for_generate['input_ids'])
                                                          model_kwargs=tokenized)

NameError: name 'tokenized' is not defined

In [136]:
decoder_input_ids = example[1]

In [138]:
help(model._prepare_decoder_input_ids_for_generation)

# decoder_input_ids = model._prepare_decoder_input_ids_for_generation(8, model_input_name=None, model_kwargs=args_for_generate)
# decoder_input_ids
ecco_model._get_embeddings(decoder_input_ids['input_ids'])

Help on method _prepare_decoder_input_ids_for_generation in module transformers.generation.utils:

_prepare_decoder_input_ids_for_generation(batch_size: int, model_input_name: str, model_kwargs: Dict[str, torch.Tensor], decoder_start_token_id: int = None, bos_token_id: int = None, device: torch.device = None) -> Tuple[torch.LongTensor, Dict[str, torch.Tensor]] method of transformers.models.t5.modeling_t5.T5ForConditionalGeneration instance
    Prepares `decoder_input_ids` for generation with encoder-decoder models



(tensor([[[  3.0951,   6.3341,   0.7727,  ...,  -1.1626,  -8.7935,  -0.7611],
          [  0.1786,   8.4092,  -2.1375,  ...,   3.3130,   1.0423,   8.3537],
          [  0.2321,  -0.1631,  -0.5348,  ...,  -1.1929,  -1.9061,  -4.5645],
          ...,
          [ -8.5768,  10.7845,  -4.7394,  ...,   0.3884, -13.4469,   5.3080],
          [  0.2321,  -0.1631,  -0.5348,  ...,  -1.1929,  -1.9061,  -4.5645],
          [ 12.9795,   5.8975,  12.4281,  ...,   4.4068, -21.2082,   9.4508]]],
        device='cuda:0', grad_fn=<UnsafeViewBackward0>),
 tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.]]], device='cuda:0', requires_grad=True))

In [6]:
args_for_generate = {'max_length': 1024, 'min_length': 128, 'early_stopping': True}
args_for_generate

{'max_length': 1024, 'min_length': 128, 'early_stopping': True}

In [8]:
# output = ecco_model.generate(text, max_length=512, do_sample=False, generate_kwargs=args_for_generate, attribution=['ig'])
# output.primary_attributions(attr_method='ig', ignore_tokens=[0,1,2,3,4,5,6,43,44])
#ecco_model.generate(text, max_length=512)  # we need the kwargs for the custom generate....

ecco_model.generate(text, max_new_tokens=128, do_sample=True, generate=13)

<IPython.core.display.Javascript object>

In [1]:
# tokenized = tokenizer('tito', max_length=50, truncation=True, return_tensors='pt')
# output = model.generate(**tokenized)
# help(output)

In [10]:
output = ecco_model.generate(f"TITO: {text} TITO_END", generate=1, do_sample=False, attribution=['ig'])
output.primary_attributions(attr_method='ig', ignore_tokens=[0,1,2,3,4,5,6,43,44])

<IPython.core.display.Javascript object>

In [53]:
inputs = ecco_model.tokenizer([text], return_tensors="pt")
output = ecco_model(inputs)

nmf_1 = output.run_nmf(n_components=8) 

nmf_1.explore()

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

In [2]:
#lm = ecco.from_pretrained('t5-small')

# MLM Explainability

https://github.com/hila-chefer/Transformer-MM-Explainability

PROBABLY NOT

BETTER BUT UNLIKELY TO GET TO THIS POINT

https://github.com/PAIR-code/lit/tree/main

may need these for that step...

!pip install lit-nlp rouge-score shap transformers